[![Colab Badge Link](https://img.shields.io/badge/open-in%20colab-blue)](https://colab.research.google.com/github/Glasgow-AI4BioMed/tutorials/blob/main/candidate_generation_for_entity_linking.ipynb)

# Candidate Generation for Entity Linking

This notebook demonstrates a few ways that you can find candidates from a list of terms when trying to do [entity linking](https://en.wikipedia.org/wiki/Entity_linking).

## Problem Definition

We have a mention of an entity in some text (e.g. 'n-sclc'). We're trying to figure out which entity it is referring to.

In [ ]:
mention_text = 'n-sclc'

We have a knowledgebase containing lots of entities (e.g. drugs, diseases, etc). Common knowledge bases include UMLS, MeSH, Disease Ontology, etc. The knowledgebase has lots of terms with their names, and possibly with additional aliases.

Let's make up a mini-list of names from our knowledgebase that we want to search.

In [ ]:
search_space = [
    'Lung cancer',
    'NSCLC',
    'Pancreatic cancer',
    'Glioblastoma multiforme',
    'Retinoblastoma',
    'breast cancer',
    'Basal cell carcinoma',
    'Squamous cell carcinoma',
    'Fibroma'
]

**The Task:** We want an short ordered list of candidates that best match our target ('n-sclc'). *The right answer is most likely NSCLC for this*.

The candidate list can then be used in the next stage of an information extraction pipeline. We might pick the top candidate as the one we'll use. Or we might use a second stage that will use another approach to score the candidates and decide which one seems the most plausible.


## Using text distance metrics

We could use the [textdistance library](https://pypi.org/project/textdistance/) and calculate distances between our `mention_text` and every entry in the search space.

First, install it:

In [ ]:
!pip install textdistance

`textdistance` contains a few options you could try. Let's go with the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to calculate the number of edits needed to change one string into another:

In [ ]:
import textdistance

textdistance.levenshtein('abc', 'bcd')

Now iterate through every option in the search space and calculate a distance metric. We'll lowercase everything for this too.

In [ ]:
scored_candidates = []
for candidate in search_space:
  score = textdistance.levenshtein(mention_text.lower(), candidate.lower())
  scored_candidates.append( (score, candidate) )

Sort all the options and get the top five.

In [ ]:
scored_candidates = sorted(scored_candidates, key=lambda x: x[0])
scored_candidates[:5]

Cool. Our **correct** answer is scored highest.

**Note:** This may be a fairly slow method if there are a lot of comparisons.

## Character N-Grams Similarity

Another approach is to use character n-grams. 3-grams is a good option and can be done with scikit-learn. Let's create a vectorizer to do this:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='char', # Work at the character-level (not token-level)
                             ngram_range=(3,3), # Tell it to use trigrams
                             norm='l2', # Tell it to normalize the vectors
                             lowercase=True)

We then fit the vectorizer with the texts of all our possible options (the `search_space`).

In [ ]:
search_vectors = vectorizer.fit_transform(search_space)

We can see what the vectorizer is doing by applying it to some text with `transform` and then running `inverse_transform`. It gets back our trigrams:

In [ ]:
vectorizer.inverse_transform(vectorizer.transform(['cancer']))

Now we can create a vector

In [ ]:
mention_vector = vectorizer.transform([mention_text])
mention_vector

Now we want to calculate the cosine similarity between our `mention_vector` and the `search_vectors`. All the vectors have already been normalized, so the cosine similarity is a dot product as below.

In [ ]:
scores = search_vectors.dot(mention_vector.T)
scores = scores.toarray().squeeze().tolist() # Convert it to a 1D python list

Let's look at the scores. Only one of them is non-zero:

In [ ]:
scores

We can pair of the scores with all the possible options from the `search_space`.

In [ ]:
scored_candidates = list(zip(scores, search_space))

Sort them (in reverse this time as the scores are similarity)

In [ ]:
scored_candidates = sorted(scored_candidates, key=lambda x: x[0], reverse=True)

And look at the the first five:

In [ ]:
top_5 = scored_candidates[:5]
top_5

Yay, the correct answer is the top answer (NSCLC).

This is likely fairly fast. If really needed, it could be made to run even faster using appropriate indexing with a library like [pyterrier_pisa](https://github.com/terrierteam/pyterrier_pisa).

## Dense Vector Search

An alternative approach uses a transformer model. It's more heavy-duty but it doesn't rely on matching the letters of the entity names, so may be able to deal with greater differences.

It uses the [SapBERT transformer model](https://huggingface.co/cambridgeltl/SapBERT-from-PubMedBERT-fulltext). The code below loads it and puts it into a [feature-extraction pipeline](https://huggingface.co/tasks/feature-extraction) that can be used to encode text into context vectors.

In [ ]:
from transformers import pipeline

model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
extractor = pipeline("feature-extraction", model=model_name)

We can generate context vectors for all the options in our search space with:

In [ ]:
model_output = extractor(search_space)

We get vectors for every single token in every single text from the search space. We only want the first vector for each option. Let's get those:

In [ ]:
cls_vectors = [ x[0][0] for x in model_output ]

And turn them into a nice numpy array:

In [ ]:
import numpy as np

search_vectors = np.array(cls_vectors)

search_vectors.shape

There were 9 options in the search space and BERT models create vectors that are 768 wide elements wide.

Now we get the vector for our mention (n-sclc) that we are searching with:

In [ ]:
model_output = extractor(mention_text)

mention_vector = np.array(model_output[0][0])

mention_vector.shape

It's a single vector with 768 elements.

To get the scores, we calculate the dot-products:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

scores = cosine_similarity(search_vectors, mention_vector.reshape(1,-1)) # Need to do .reshape to make it a 2D matrix
scores = scores.flatten().tolist() # Get them into a 1D python list

In [ ]:
scores

Next, we group the scores with all the options from the search space and sort them. Higher is better, so we use `reverse=True` to put the best candidates at the front of the list.

In [ ]:
scored_candidates = list(zip(scores, search_space))
scored_candidates = sorted(scored_candidates, key=lambda x: x[0], reverse=True)

Check the first five:

In [ ]:
top_5 = scored_candidates[:5]
top_5

And we're pleased to see that the correct option (NSCLC) is at the top of the list. Notably, lung cancer is second in the list which may be because NSCLC is a type of lung cancer.

### Make it even faster!

If you wanted to make the dense vector search faster, you could use a library such as faiss. It can be accelerate with GPUs but we'll just go over the CPU approach here.

First install it:

In [ ]:
!pip install faiss-cpu

Create an index of the right size (for vectors with 768 elements) and add in our search vectors:

In [ ]:
import faiss

index_flat = faiss.IndexFlatIP(768)
index_flat.add(search_vectors)

Then run the search with our `mention_vector` and ask for the top five:

In [ ]:
scores, indices = index_flat.search(mention_vector.reshape(1,768), 5)

print(scores)
print(indices)

We actually get the same scores as we did before along with which index they correspond to.

We can create our usual ordered candidate list:

In [ ]:
scored_candidates = [ (score,search_space[idx]) for score,idx in zip(scores[0],indices[0]) ]

Look at the five that were returned:

In [ ]:
scored_candidates

And we see that NSCLC is again top with this method